In [ ]:
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
import torch
import pandas as pd

In [ ]:
# !git clone https://github.com/koaNam/yolov5

In [ ]:
# !pip3 install -r ./yolov5/requirements.txt

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./source/last_20210531.pt', force_reload=True)

In [ ]:
cap = cv2.VideoCapture(0)
ret, img = cap.read()
cropped_img = img
height, width, channels = img.shape

if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    for val in model(imgRGB).pandas().xyxy:
        for i in range(0, len(val.index)):
            xmin = val.iloc[i]["xmin"]
            ymin = val.iloc[i]["ymin"]
            xmax = val.iloc[i]["xmax"]
            ymax = val.iloc[i]["ymax"]
            
            if xmin < 20 or ymin < 20 or xmax > width - 20 or ymax > height - 20:
                continue
            rectangle = cv2.rectangle(img, (int(xmin)-20, int(ymin)-20), (int(xmax)+20, int(ymax)+20), (0, 255, 0), 2)
            cropped_img = img[int(ymin)-20:int(ymax+20), int(xmin)-20:int(xmax)+20]
        cv2.imshow("Crop", cropped_img)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()